<a href="https://colab.research.google.com/github/runningpoem30/TensorTitans/blob/main/Final_PS04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers datasets
!pip install accelerate -U
!pip install tensorboardX -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install  datasets

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from datasets import load_dataset
# Load the dataset "HuggingFaceH4/no_robots"
dataset1 = load_dataset('HuggingFaceH4/no_robots')
print(dataset1)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category'],
        num_rows: 9500
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category'],
        num_rows: 500
    })
})


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Model and Tokenizer initialization
model_name_or_path = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
model = BartForConditionalGeneration.from_pretrained(model_name_or_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from torch.utils.data import Dataset

# Load the dataset "HuggingFaceH4/no_robots"
dataset = load_dataset('HuggingFaceH4/no_robots')

# Select only 'prompt' and 'messages' features
dataset_subset = dataset['train'].select(range(6000))

# Extract 'prompt' and 'messages' from the dataset
train_texts = [{'prompt': ex['prompt'], 'messages': ex['messages']} for ex in dataset_subset]

# Split into training and validation datasets
train_size = int(len(train_texts) * 0.9)
train_dataset = train_texts[:train_size]
val_dataset = train_texts[train_size:]

# Define a custom dataset class
class NoRobotsDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):  # Reduced max_length
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        prompt = item['prompt']
        messages = item['messages']

        # Ensure inputs are of valid type
        if isinstance(prompt, list):
            prompt = ' '.join(prompt)

        # Extract text from dictionaries in messages
        if isinstance(messages, list):
            messages = ' '.join([msg['content'] for msg in messages if isinstance(msg, dict) and 'content' in msg])

        # Tokenize inputs and targets
        inputs = self.tokenizer(prompt, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")
        targets = self.tokenizer(messages, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")

        inputs = {key: val.squeeze() for key, val in inputs.items()}
        targets = {key: val.squeeze() for key, val in targets.items()}

        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': targets['input_ids']
        }

# Load a smaller tokenizer and model variant
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")  # Using BART-Base

# Prepare the training and validation datasets using custom dataset class
max_length = 128  # Reduced max_length
train_dataset = NoRobotsDataset(train_dataset, tokenizer, max_length=max_length)
val_dataset = NoRobotsDataset(val_dataset, tokenizer, max_length=max_length)

# Load a smaller model variant
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")  # Using BART-Base

# Move the model to CPU
device = torch.device("cpu")
model.to(device)

# TrainingArguments for fine-tuning on CPU with reduced batch size and gradient accumulation steps
training_args = TrainingArguments(
    output_dir="./no_robots_finetuned",
    per_device_train_batch_size=1,            # Reduced batch size to 1
    gradient_accumulation_steps=2,            # Accumulate gradients over 2 steps (adjust as needed)
    per_device_eval_batch_size=1,
    eval_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
    num_train_epochs=1,  # Only training for 1 epoch to reduce time
    report_to="tensorboard",
    use_cpu=True,
)

# Initialize Trainer and start fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./no_robots_model_finetuned")

# Save the tokenizer
tokenizer.save_pretrained("./no_robots_model_finetuned")

# Save the generation configuration
from transformers import GenerationConfig

generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
generation_config.save_pretrained("./no_robots_model_finetuned")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
0,3.220500,2.524121


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load fine-tuned model and tokenizer
model_path = "./no_robots_model_finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [ ]:
import torch

# Define function for chatbot interaction
def chatbot_interface():
    print("Welcome to the Chatbot! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Bye! Have a great day.")
            break

        # Tokenize input text
        inputs = tokenizer(user_input, return_tensors="pt")

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                inputs['input_ids'],
                max_length=50,
                num_beams=5,
                early_stopping=True,
                decoder_start_token_id=model.config.decoder_start_token_id,
                bos_token_id=model.config.bos_token_id,
                pad_token_id=model.config.pad_token_id,
                eos_token_id=model.config.eos_token_id,
            )

        # Decode and print response
        chatbot_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Chatbot: {chatbot_response}")

# Run the chatbot interface
chatbot_interface()


Welcome to the Chatbot! Type 'exit' to end the conversation.
You: write a note on taking leave
Chatbot: write a note on taking leave
You: exit
Chatbot: Bye! Have a great day.
